In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_estimator as tfe

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_csv("./data/cal_housing_clean.csv")

In [3]:
df.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41,880,129,322,126,8.3252,452600
1,21,7099,1106,2401,1138,8.3014,358500
2,52,1467,190,496,177,7.2574,352100
3,52,1274,235,558,219,5.6431,341300
4,52,1627,280,565,259,3.8462,342200


In [4]:
y_val= df["medianHouseValue"]

x_data=df.drop("medianHouseValue",axis=1)

In [5]:
X_train, X_eval,y_train,y_eval=train_test_split(x_data,y_val,test_size=0.3,random_state=101)

In [6]:
scaler_model = MinMaxScaler()

scaler_model.fit(X_train)



X_train=pd.DataFrame(scaler_model.transform(X_train),columns=X_train.columns,index=X_train.index)



scaler_model.fit(X_eval)



X_eval=pd.DataFrame(scaler_model.transform(X_eval),columns=X_eval.columns,index=X_eval.index)

C:\Users\OrsonKong\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\OrsonKong\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [7]:
#Creating Feature Columns

feat_cols=[]

for cols in df.columns[:-1]:

    column=tf.feature_column.numeric_column(cols)

    feat_cols.append(column)

    

print(feat_cols)

[NumericColumn(key='housingMedianAge', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='totalRooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='totalBedrooms', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='population', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='households', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='medianIncome', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [16]:
#The estimator model

model=tf.estimator.DNNRegressor(hidden_units=[6,10,6],feature_columns=feat_cols, activation_fn=tf.keras.layers.LSTM)



#the input function

input_func=tfe.estimator.inputs.pandas_input_fn(X_train,y_train,batch_size=10,num_epochs=1000,shuffle=True)

W0328 17:31:16.633782 13308 estimator.py:1799] Using temporary folder as model directory: C:\Users\ORSONK~1\AppData\Local\Temp\tmp325t780m


In [17]:
#Training the model

model.train(input_fn=input_func,steps=1000)

ValueError: ('Could not interpret activation function identifier:', <property object at 0x0000020517410E58>)

In [36]:
#Evaluating the model

train_metrics=model.evaluate(input_fn=input_func,steps=1000)

In [37]:
#Now to predict values we do the following

pred_input_func=tfe.estimator.inputs.pandas_input_fn(x=X_eval,y=y_eval,batch_size=10,num_epochs=1,shuffle=False)

preds=model.predict(input_fn=pred_input_func)



predictions=list(preds)

final_pred=[]

for pred in predictions:

    final_pred.append(pred["predictions"])

In [38]:
len(final_pred)

6192

In [39]:
final_pred[:10]

[array([208158.66], dtype=float32),
 array([238296.97], dtype=float32),
 array([207592.72], dtype=float32),
 array([164688.95], dtype=float32),
 array([220743.44], dtype=float32),
 array([193928.08], dtype=float32),
 array([218088.62], dtype=float32),
 array([192967.45], dtype=float32),
 array([189859.83], dtype=float32),
 array([143863.28], dtype=float32)]

In [42]:
np.mean(final_pred)

195732.23

In [43]:
np.mean(final_prev)

138072.53

In [40]:
final_prev[:10] #without scaling data

[array([90901.71], dtype=float32),
 array([53624.027], dtype=float32),
 array([79372.19], dtype=float32),
 array([88207.77], dtype=float32),
 array([161773.83], dtype=float32),
 array([207049.25], dtype=float32),
 array([119842.25], dtype=float32),
 array([109163.125], dtype=float32),
 array([113261.03], dtype=float32),
 array([5385.719], dtype=float32)]